In [6]:
import pandas as pd
import numpy as np
import re
from util import string_to_yuan

df = pd.read_json('./sample_p4.txt', encoding='gb18030', lines=True)
df.dropna(axis=0, subset=['content'], inplace=True)
print('非空:', len(df))
df = df.iloc[np.bitwise_and(
    df['trial_round'].values != '其他', df['trial_round'].values != '刑罚变更'), :]
print('非其他，非刑法变更:', len(df))


def is_valid(t):
    if t.find('撤回上诉') != -1:
        return False
    if t.find('重新审判') != -1:
        return False
    if t.find("申诉理由不能成立") != -1:
        return False
    return True


df = df.iloc[df['content'].apply(is_valid).values, :]
print('非撤回上诉:', len(df))

fact = []


def get_money(t: str):
    ret = []

    m = re.findall(f"(?<=追缴赃款).*元", t, re.M)
    ret.extend(m)

    m = re.findall(f"(?<=违法所得).*元", t, re.M)
    ret.extend(m)

    m = re.findall(f"(?<=所得赃款).*元", t, re.M)
    ret.extend(m)

    m = re.findall(f"(?<=收受的).*?(?=赃款依法予以没收)", t, re.M)
    ret.extend(m)

    m = re.findall(f"(?<=收受的).*?(?=上缴)", t, re.M)
    ret.extend(m)
    
    rret = []
    for t in ret:
        if re.search('[，。；,.、]',t):
            continue
        rret.append(t)
    ret = rret
    
    if len(ret)==0:
        m = re.findall("(?<=共计).*元",t,re.M)
        ret.extend(m)
        
        m = re.findall("(?<=总计).*元",t,re.M)
        ret.extend(m)
    
    rret = []
    for t in ret:
        if re.search('[，。；,.、]',t):
            continue
        rret.append(t)
    ret = rret
    
    if len(ret) == 0:
        m = re.findall(f"[\d零一二三四五六七八九十百千万]*元", t, re.M)
        ret.extend(m)
            
    return ret

def money_clear(m:str):
    try:
        return re.search('[\d零一二三四五六七八九十百千万]*元',m).group(0)
    except:
        return m

def select_fact(t:str):
    ret = []
    facts = re.findall("(?<=认证的下列证据予以证实：\n).*?(?=以上)",t,re.DOTALL)
    for fact in facts:
        f0 = fact.split('\n')[0]
        ret.append(f0)
    return ''.join(ret)
   

for idx, row in df.iterrows():
    text = row['content']
    bg = text.find("经审理查明")
    ed = text.find("本院认为")
    text = text[bg+6:ed]
    fact.append(text)

df.insert(2, 'fact', fact)

money = []
cnt = 1

for idx, row in df.iterrows():
    content = row['content']
    fact = select_fact(content)
    if fact:
        m = get_money(fact)
    else:
        m = get_money(content)
        
    # if(len(m) == 0):
    #     cnt += 1
    #     print(row['content'])

    money.append(m)

print('金额提取成功比例',np.sum([len(m)!=0 for m in money])/len(money))

def sum_money(m):
    ret = 0
    for t in m:
        t = money_clear(t)
        try:
            ret += string_to_yuan(t)
        except:
            print(t)
            break
    return ret
print([sum_money(m) for m in money])
        

# cnt= 0
# for idx,m in enumerate(money):
#     if len(m)==0:
#         print(idx)
#         print(m)
#         print(df.iloc[idx]['content'])
#         cnt+=1
#     if cnt>10:
#         break


非空: 951
非其他，非刑法变更: 624
非撤回上诉: 564
金额提取成功比例 0.9680851063829787
贿赂属应
四五千元
七八万元
三四百万元
[45000, 17251.0, 120000.0, 30000, 125000.0, 81980000.0, 40000, 124000.0, 540000.0, 81000.0, 335000, 4.0, 130000, 1795000.0, 4310000.0, 480000.0, 70000.0, 69700.0, 430000.0, 202000.0, 237000.0, 10440000.0, 103750.0, 28083.0, 250372.0, 399800.0, 1080000.0, 2339000.0, 74000.0, 950000.0, 20000.0, 30000, 30500.0, 200800.0, 0, 80000, 96201.0, 304000.0, 628335.0, 178000.0, 187000.0, 29200.0, 1530000.0, 27000.0, 59000.0, 14448, 260000.0, 920000.0, 294800.0, 60000.0, 217000.0, 23850.0, 80000.0, 35500, 960000.0, 449000.0, 50000, 180000.0, 180000.0, 220000.0, 40800.0, 40000.0, 1800000.0, 37000.0, 350000.0, 90000.0, 50000.0, 630000.0, 20000.0, 468450.0, 20000.0, 740000.0, 20000.0, 6713520.0, 400000.0, 130000.0, 504960.0, 50000.0, 271600.0, 135608.0, 100000, 320000.0, 1610000.0, 2757450.0, 280000.0, 218546.0, 50000.0, 1170000.0, 147300.0, 18000.0, 460000.0, 500.0, 95.0, 200000.0, 35000.0, 503740.0, 2200000.0, 500000.